In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gc

print("Starting fraud detection model training...")

# Step 1: Load the Data
# Ensure these paths are correct for your environment
try:
    train_identity = pd.read_csv(r'/content/train_identity.csv')
    train_transaction = pd.read_csv(r"/content/train_transaction.csv")
    test_identity = pd.read_csv(r"/content/test_identity.csv")
    test_transaction = pd.read_csv(r"/content/test_transaction.csv")
    sample_submission = pd.read_csv(r"/content/sample_submission.csv")
    print("Data loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading data: {e}. Please ensure the CSV files are in the specified '/content/' directory.")
    exit() # Exit if data files are not found

# Step 1.1: Fix column names in test_identity
# This ensures consistency in column names for merging.
test_identity.columns = [col.replace('id-', 'id_') for col in test_identity.columns]

# Step 1.2: Combine the datasets
# Merge transaction and identity data for both train and test sets.
# 'left' merge is used to keep all transaction records and add identity info where available.
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
print("Train and test datasets merged.")

# Step 1.3: Optimize memory by downcasting numerical columns
# This function reduces memory usage by converting numerical columns to smaller data types.
def downcast_df(df):
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

train = downcast_df(train)
test = downcast_df(test)
print("Numerical columns downcasted for memory optimization.")

# Store TransactionID for test set alignment later
test_ids = test['TransactionID'].copy()

# Clean up memory
# Explicitly delete original dataframes to free up memory.
del train_identity, train_transaction, test_identity, test_transaction
gc.collect() # Garbage collection

print(f"Train shape after merge and downcast: {train.shape}, Test shape: {test.shape}")

# Step 2: Preprocessing
# 2.1: Remove features with high missing values (>60%)
# Columns with more than 60% missing values are typically not very useful and are removed.
missing_percent = train.isnull().mean()
high_missing_cols = missing_percent[missing_percent > 0.6].index.tolist()
# Ensure 'TransactionID' is not accidentally removed if it meets the criteria
if 'TransactionID' in high_missing_cols:
    high_missing_cols.remove('TransactionID')
train.drop(columns=high_missing_cols, inplace=True)
test.drop(columns=high_missing_cols, inplace=True)
print(f"Removed {len(high_missing_cols)} columns with >60% missing values.")
print(f"Train shape after dropping high missing cols: {train.shape}, Test shape: {test.shape}")


# 2.2: Define categorical and numerical columns for processing
# Identify columns that are likely categorical or numerical based on their data types and context.
# Ensure 'isFraud' and 'TransactionID' are excluded from numerical feature lists as they are target/identifiers.
all_cols = set(train.columns)
# Explicitly defined categorical columns
initial_categorical_cols = ['ProductCD', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain',
                            'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9']
initial_categorical_cols = [col for col in initial_categorical_cols if col in all_cols]

# Identify columns that are objects and not in initial categorical list (potentially more categories or other objects)
object_cols = train.select_dtypes(include='object').columns.tolist()
additional_categorical_cols = [col for col in object_cols if col not in initial_categorical_cols]
# Union of both lists
categorical_cols = list(set(initial_categorical_cols + additional_categorical_cols))

# Numerical columns: all remaining numeric types, excluding target and ID.
numerical_cols = train.select_dtypes(include=['float32', 'float64', 'int8', 'int16', 'int32']).columns.tolist()
if 'isFraud' in numerical_cols:
    numerical_cols.remove('isFraud')
if 'TransactionID' in numerical_cols:
    numerical_cols.remove('TransactionID')
# Remove any numerical columns that were also identified as categorical (e.g., if LabelEncoder was applied early)
numerical_cols = [col for col in numerical_cols if col not in categorical_cols]

print(f"Identified {len(categorical_cols)} categorical columns and {len(numerical_cols)} numerical columns.")


# 2.3: Handle missing values
# Impute numerical features with their median and categorical features with 'missing' placeholder.
for col in numerical_cols:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(test[col].median())
    if train[col].isnull().any() or test[col].isnull().any(): # Double check after median fill
        print(f"Warning: Missing values still exist in numerical column {col} after median fill.")

for col in categorical_cols:
    train[col] = train[col].fillna('missing').astype(str)
    test[col] = test[col].fillna('missing').astype(str)
    if train[col].isnull().any() or test[col].isnull().any(): # Double check after 'missing' fill
        print(f"Warning: Missing values still exist in categorical column {col} after 'missing' fill.")
print("Missing values handled for numerical and categorical columns.")


# 2.4: Encode categorical features using LabelEncoder
# LabelEncoder converts categorical text data into numerical labels, suitable for embeddings.
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    # Combine train and test data for fitting to ensure consistent encoding across both datasets.
    combined = pd.concat([train[col], test[col]], axis=0)
    le.fit(combined.astype(str)) # Fit on string type
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))
    label_encoders[col] = le
    # Store the number of unique classes for embedding layer dimensions
    # +1 for 'missing' or any new unseen categories if applicable, though LabelEncoder handles this by assigning largest int.
    train[col] = train[col].astype(int)
    test[col] = test[col].astype(int)
print("Categorical features encoded using LabelEncoder.")

# Step 3: Feature Engineering
# 3.1: Time-based feature
# Extract 'hour' from 'TransactionDT' for potential daily patterns.
train['hour'] = (train['TransactionDT'] // 3600) % 24
test['hour'] = (test['TransactionDT'] // 3600) % 24
numerical_cols.append('hour') # Add 'hour' to numerical features
print("Engineered 'hour' feature.")


# 3.2: Log transform TransactionAmt
# Apply log transformation to TransactionAmt to reduce skewness and handle large ranges.
train['LogTransactionAmt'] = np.log1p(train['TransactionAmt'])
test['LogTransactionAmt'] = np.log1p(test['TransactionAmt'])
numerical_cols.append('LogTransactionAmt') # Add 'LogTransactionAmt' to numerical features
print("Log transformed 'TransactionAmt'.")


# 3.3: Transaction frequency for card1
# Create a frequency-based feature for 'card1' (a common card identifier).
if 'card1' in train.columns:
    freq_map = train['card1'].value_counts().to_dict()
    train['card1_freq'] = train['card1'].map(freq_map)
    test['card1_freq'] = test['card1'].map(freq_map)
    # Fill NaN values (for card1 values present in test but not train) with 0 frequency.
    train['card1_freq'] = train['card1_freq'].fillna(0).astype(int)
    test['card1_freq'] = test['card1_freq'].fillna(0).astype(int)
    numerical_cols.append('card1_freq') # Add 'card1_freq' to numerical features
    print("Engineered 'card1_freq' feature.")


# Ensure the final numerical_cols and categorical_cols lists contain only columns that exist in the dataframe
numerical_cols = [col for col in numerical_cols if col in train.columns and col not in ['TransactionID', 'isFraud']]
categorical_cols = [col for col in categorical_cols if col in train.columns and col not in ['TransactionID', 'isFraud']]

# Step 4: Feature Preparation for Transformer
# 4.1: Define feature set for the model
# The feature set consists of all identified numerical and categorical columns.
features = numerical_cols + categorical_cols
X = train[features]
y = train['isFraud']
X_test_full = test[features] # The full test set for final predictions

print(f"Final features selected for model: {features}")
print(f"Number of numerical features: {len(numerical_cols)}")
print(f"Number of categorical features: {len(categorical_cols)}")

# 4.2: Scale numerical features
# Standardize numerical features to have zero mean and unit variance. This is crucial for models like Transformers.
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])
X_test_full[numerical_cols] = scaler.transform(X_test_full[numerical_cols])
print("Numerical features scaled using StandardScaler.")

# Step 4.5: Feature Importance with L1-Regularized Logistic Regression
# L1 regularization (Lasso) can drive less important feature coefficients to zero, aiding in feature selection and importance.
print("\n--- Calculating Feature Importance using L1-Regularized Logistic Regression ---")
# Use a copy of X for Logistic Regression to avoid SettingWithCopyWarning
X_l1_reg = X.copy()
log_reg_l1 = LogisticRegression(penalty='l1', solver='liblinear', random_state=42, C=0.1) # C is inverse of regularization strength
log_reg_l1.fit(X_l1_reg, y)

feature_importance = pd.DataFrame({
    'Feature': features,
    'Coefficient': log_reg_l1.coef_[0]
})
feature_importance['Absolute_Coefficient'] = np.abs(feature_importance['Coefficient'])
feature_importance = feature_importance.sort_values(by='Absolute_Coefficient', ascending=False)
print("Top 20 Feature Importances (by absolute coefficient):")
print(feature_importance.head(20).to_string(index=False))
print("--- End Feature Importance ---")


# 4.3: SMOTE for handling class imbalance
# SMOTE (Synthetic Minority Over-sampling Technique) creates synthetic samples of the minority class.
# This helps prevent the model from being biased towards the majority class.
print("Applying SMOTE to address class imbalance...")
smote = SMOTE(random_state=42, sampling_strategy=1.0) # Full oversampling to balance classes
X_resampled, y_resampled = smote.fit_resample(X, y)
print(f"Original class distribution: {y.value_counts()}")
print(f"Resampled class distribution: {y_resampled.value_counts()}")

# 4.4: Create train/val/test split (70/15/15)
# Split the resampled data into training, validation, and test sets.
# Validation set is used for monitoring training performance and early stopping.
X_train_resampled, X_temp_resampled, y_train_resampled, y_temp_resampled = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)
X_val_resampled, X_test_resampled, y_val_resampled, y_test_resampled = train_test_split(
    X_temp_resampled, y_temp_resampled, test_size=0.5, random_state=42, stratify=y_temp_resampled
)

print(f"Resampled Train set shape: {X_train_resampled.shape}, Validation set shape: {X_val_resampled.shape}, Test set shape: {X_test_resampled.shape}")

# Prepare inputs for TensorFlow model (convert to numpy arrays of appropriate type)
X_train_num = X_train_resampled[numerical_cols].values.astype(np.float32)
X_val_num = X_val_resampled[numerical_cols].values.astype(np.float32)
X_test_num = X_test_resampled[numerical_cols].values.astype(np.float32)
X_test_full_num = X_test_full[numerical_cols].values.astype(np.float32)


# Step 5: Build Transformer Model
# This section defines the Transformer architecture for tabular data.
# It uses embedding layers for categorical features and combines them with numerical features.
# A custom Transformer block is defined for multi-head self-attention.

def create_transformer_model(
    num_numerical_features,
    categorical_features_info, # List of (col_name, num_unique_values) for embeddings
    embedding_dim=32,
    num_heads=4,
    ff_dim=128,
    num_transformer_blocks=2,
    mlp_units=[256, 128],
    dropout_rate=0.2
):
    # Input for numerical features
    numerical_input = keras.Input(shape=(num_numerical_features,), name="numerical_input")
    numerical_features = numerical_input

    # Inputs for categorical features and their embeddings
    categorical_inputs = [] # This will hold the Keras Input layers for the model's inputs
    all_feature_embeddings_for_stack = [] # This will hold the flattened embedding tensors for stacking

    # Add numerical feature projection to the list of features for stacking
    numerical_feature_projected = layers.Dense(embedding_dim, activation='relu', name="numerical_projection")(numerical_input)
    all_feature_embeddings_for_stack.append(numerical_feature_projected)

    # Process categorical features: create input, embedding, and add to lists
    for col_name, num_unique_values in categorical_features_info:
        cat_input = keras.Input(shape=(1,), name=f"cat_input_{col_name}", dtype=tf.int32)
        categorical_inputs.append(cat_input) # Add to the list of model inputs

        embedding = layers.Embedding(
            input_dim=num_unique_values, # Corrected: Use the exact number of unique classes from LabelEncoder
            output_dim=embedding_dim,
            name=f"embedding_{col_name}"
        )(cat_input)
        all_feature_embeddings_for_stack.append(layers.Flatten()(embedding)) # Add flattened embedding to list for stacking

    # Stack all projected features (numerical and categorical embeddings) to create a "sequence" for the transformer.
    # Shape: (batch_size, num_features, embedding_dim)
    transformer_input = layers.Lambda(lambda x: tf.stack(x, axis=1))(all_feature_embeddings_for_stack)

    # Add positional embeddings (learned)
    num_tokens = 1 + len(categorical_features_info) # Numerical features treated as one token, plus one for each categorical
    positional_embedding_layer = layers.Embedding(num_tokens, embedding_dim)
    positions = tf.range(start=0, limit=num_tokens, delta=1)
    positional_embeddings = positional_embedding_layer(positions)
    x = transformer_input + positional_embeddings # Add positional embeddings to input features

    # Transformer Blocks
    for _ in range(num_transformer_blocks):
        # Multi-Head Attention
        attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(x, x)
        attn_output = layers.Dropout(dropout_rate)(attn_output)
        attn_output = layers.LayerNormalization(epsilon=1e-6)(x + attn_output) # Add & Norm

        # Feed-Forward Network
        ffn_output = layers.Dense(ff_dim, activation="relu")(attn_output)
        ffn_output = layers.Dense(embedding_dim)(ffn_output) # Project back to embedding_dim
        ffn_output = layers.Dropout(dropout_rate)(ffn_output)
        x = layers.LayerNormalization(epsilon=1e-6)(attn_output + ffn_output) # Add & Norm

    # Global Average Pooling or Flatten for classification head
    x = layers.GlobalAveragePooling1D()(x) # Shape: (batch_size, embedding_dim)

    # MLP for classification
    for units in mlp_units:
        x = layers.Dense(units, activation="relu")(x)
        x = layers.Dropout(dropout_rate)(x)

    # Output layer
    output = layers.Dense(1, activation="sigmoid", name="output")(x)

    # Define the model with all inputs
    model = keras.Model(inputs=[numerical_input] + categorical_inputs, outputs=output)
    return model

# Prepare categorical feature info for model creation
categorical_features_info = []
for col in categorical_cols:
    # IMPORTANT FIX: Use len(le.classes_) to get the exact number of unique categories
    # that the LabelEncoder was fitted on, ensuring correct input_dim for Embedding.
    num_unique_values = len(label_encoders[col].classes_)
    categorical_features_info.append((col, num_unique_values))


# Instantiate and compile the Transformer model
num_numerical_features = len(numerical_cols)
transformer_model = create_transformer_model(
    num_numerical_features=num_numerical_features,
    categorical_features_info=categorical_features_info
)

transformer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=[
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        'accuracy'
    ]
)

transformer_model.summary()
print("Transformer model built and compiled.")

# Prepare data for model training (list of arrays for inputs)
# Categorical features need to be passed as separate inputs for embedding layers
# Ensure each categorical input array has shape (batch_size, 1) to match keras.Input(shape=(1,))
X_train_cat = [X_train_resampled[col].values.astype(np.int32).reshape(-1, 1) for col in categorical_cols]
X_val_cat = [X_val_resampled[col].values.astype(np.int32).reshape(-1, 1) for col in categorical_cols]
X_test_cat = [X_test_resampled[col].values.astype(np.int32).reshape(-1, 1) for col in categorical_cols]
X_test_full_cat = [X_test_full[col].values.astype(np.int32).reshape(-1, 1) for col in categorical_cols]

train_inputs = [X_train_num] + X_train_cat
val_inputs = [X_val_num] + X_val_cat
test_inputs = [X_test_num] + X_test_cat
full_test_inputs = [X_test_full_num] + X_test_full_cat


# Step 6: Train Transformer Model
print("\n--- Training Transformer Model ---")
# Use early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=10,
    restore_best_weights=True,
    mode='max'
)

history = transformer_model.fit(
    train_inputs,
    y_train_resampled,
    validation_data=(val_inputs, y_val_resampled),
    epochs=50, # Set a reasonably high number, early stopping will manage
    batch_size=1024,
    callbacks=[early_stopping],
    verbose=1
)
print("Transformer model training complete.")

# Step 7: Evaluate on Test Set and Train Set
# Function to calculate all required metrics including G-mean
def calculate_metrics(y_true, y_pred_probs, threshold=0.5, name=""):
    y_pred = (y_pred_probs >= threshold).astype(int)

    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    auc = roc_auc_score(y_true, y_pred_probs)
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate Sensitivity (Recall) and Specificity
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    g_mean = np.sqrt(sensitivity * specificity)

    print(f"\n{name} Set Performance (Threshold={threshold}):")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"G-Mean (Sensitivity-Specificity): {g_mean:.4f}")
    return precision, recall, f1, auc, accuracy, specificity, g_mean


# Evaluate on Train Set
print("\n--- Evaluating on Train Set ---")
train_probs = transformer_model.predict(train_inputs).flatten()
train_metrics = calculate_metrics(y_train_resampled, train_probs, name="Train")

# Evaluate on Test Set
print("\n--- Evaluating on Test Set ---")
test_probs = transformer_model.predict(test_inputs).flatten()
test_metrics = calculate_metrics(y_test_resampled, test_probs, name="Test")

# Step 8: Generate Predictions for Submission
print("\n--- Generating Submission File ---")
# Predict probabilities on the original, un-split test data
test_full_probs = transformer_model.predict(full_test_inputs).flatten()

# Create a DataFrame with TransactionID and predictions
test_pred_df = pd.DataFrame({
    'TransactionID': test_ids,
    'isFraud': test_full_probs
})

# Merge with sample_submission to align predictions
# Fill any missing TransactionIDs from sample_submission that might not be in our test_pred_df
# (though with left merge and copying test_ids, this should be minimal).
# Use mean if any prediction is missing, though probabilities should always be generated.
sample_submission = sample_submission.merge(test_pred_df, on='TransactionID', how='left', suffixes=('', '_pred'))
sample_submission['isFraud'] = sample_submission['isFraud_pred'].fillna(test_full_probs.mean()) # Fallback
sample_submission = sample_submission[['TransactionID', 'isFraud']]

# Save submission file
submission_filename = 'submission_transformer_model.csv'
sample_submission.to_csv(submission_filename, index=False)
print(f"\nSubmission file '{submission_filename}' created successfully!")
print("Model training and prediction complete.")


Starting fraud detection model training...
Data loaded successfully.
Train and test datasets merged.
Numerical columns downcasted for memory optimization.
Train shape after merge and downcast: (7900, 434), Test shape: (5511, 433)
Removed 244 columns with >60% missing values.
Train shape after dropping high missing cols: (7900, 190), Test shape: (5511, 189)
Identified 13 categorical columns and 175 numerical columns.
Missing values handled for numerical and categorical columns.
Categorical features encoded using LabelEncoder.
Engineered 'hour' feature.
Log transformed 'TransactionAmt'.
Engineered 'card1_freq' feature.
Final features selected for model: ['TransactionDT', 'TransactionAmt', 'card1', 'card2', 'card3', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D10', 'D15', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V

/tmp/ipython-input-1-3043268185.py:139: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train['hour'] = (train['TransactionDT'] // 3600) % 24
/tmp/ipython-input-1-3043268185.py:140: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test['hour'] = (test['TransactionDT'] // 3600) % 24
/tmp/ipython-input-1-3043268185.py:147: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.

Top 20 Feature Importances (by absolute coefficient):
          Feature  Coefficient  Absolute_Coefficient
            card6    -0.841133              0.841133
              C12     0.688827              0.688827
        ProductCD    -0.449612              0.449612
            addr2    -0.434797              0.434797
LogTransactionAmt     0.407318              0.407318
              V67     0.278208              0.278208
              V83     0.274960              0.274960
              V12     0.260302              0.260302
            card3     0.259873              0.259873
              V20    -0.249863              0.249863
             V287     0.191170              0.191170
              V70    -0.175225              0.175225
       card1_freq    -0.166927              0.166927
             V120     0.162879              0.162879
              V63    -0.158228              0.158228
               C7    -0.156715              0.156715
               M4    -0.150241              0

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cat_input_M1        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_M3        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_ProductCD │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_addr2     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_M6        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_M4        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_M2        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_card5     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_card6     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_M5        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_addr1     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_card4     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cat_input_P_emaild… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ numerical_input     │ (None, 178)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_M1        │ (None, 1, 32)     │         64 │ cat_input_M1[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_M3        │ (None, 1, 32)     │         96 │ cat_input_M3[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_ProductCD │ (None, 1, 32)     │        160 │ cat_input_Produc

 Total params: 104,225 (407.13 KB)

 Trainable params: 104,225 (407.13 KB)

 Non-trainable params: 0 (0.00 B)

Transformer model built and compiled.

--- Training Transformer Model ---
Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 24s 942ms/step - accuracy: 0.5831 - auc: 0.6104 - loss: 0.6801 - precision: 0.6019 - recall: 0.4821 - val_accuracy: 0.6845 - val_auc: 0.7740 - val_loss: 0.6445 - val_precision: 0.7423 - val_recall: 0.5653
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 588ms/step - accuracy: 0.6776 - auc: 0.7452 - loss: 0.6404 - precision: 0.6964 - recall: 0.6293 - val_accuracy: 0.7308 - val_auc: 0.8164 - val_loss: 0.5958 - val_precision: 0.7638 - val_recall: 0.6681
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 607ms/step - accuracy: 0.7367 - auc: 0.8089 - loss: 0.5912 - precision: 0.7576 - recall: 0.6939 - val_accuracy: 0.7610 - val_auc: 0.8539 - val_loss: 0.5356 - val_precision: 0.7844 - val_recall: 0.7200
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 708ms/step - accuracy: 0.7804 - auc: 0.8567 - loss: 0.5296 - precision: 0.7888 - recall: 0.7559 - val_accuracy: 0.8094 - val_auc: 0.8935 - val_loss: 0.4630 